# Experiment 2: Dirichlet Parameter (Alpha) Experiments
This notebook contains templates for Dirichlet parameter (alpha) experiments with different configurations:
- Tasks: Regression, Binary Classification, and Image Classification
- Network Types: Centralised and Decentralised
- Network Size: 10 (fixed)
- Dirichlet alpha values: 0.4, 0.8, 10

In [1]:
import sys
sys.path.append("../..")
import torch
import torch.nn as nn
from torch.optim import SGD
import asyncio
import nest_asyncio
from flwr_datasets.partitioner import DirichletPartitioner
from moxi import create_experiment

# Initialize asyncio for Jupyter
nest_asyncio.apply()

/home/sgtec/miniconda3/envs/moxifl/lib/python3.11/site-packages/requests/__init__.py:86: RequestsDependencyWarning: Unable to find acceptable character detection dependency (chardet or charset_normalizer).
  warnings.warn(


## Model Definitions
Define the models for each task type

In [2]:
# Regression Model
class RegressionModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.act_func = nn.ReLU()
        self.layer1 = nn.Linear(3, 24)  # 3 input features
        self.layer2 = nn.Linear(24, 1)  # 1 output
        
    def forward(self, x):
        x = self.act_func(self.layer1(x))
        x = self.layer2(x)
        return x

# Binary Classification Model
class BinaryClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Linear(30, 64)
        self.act_func = nn.ReLU()
        self.layer2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.act_func(self.layer1(x))
        x = self.layer2(x)
        return x

# Image Classification Model
class CNNModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.fc1 = nn.Linear(128 * 4 * 4, 512)
        self.fc2 = nn.Linear(512, 10)
        self.maxpool = nn.MaxPool2d(2, 2)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        x = self.activation(self.maxpool(self.conv1(x)))
        x = self.activation(self.maxpool(self.conv2(x)))
        x = self.activation(self.maxpool(self.conv3(x)))
        x = x.view(-1, 128 * 4 * 4)
        x = self.activation(self.fc1(x))
        x = self.fc2(x)
        return x

## Experiment Runner Function

In [3]:
async def execute_experiment(flnw, num_rounds:int, epochs_per_round:int, experiment_name:str):
    """
    Execute a federated learning experiment.
    
    Args:
        flnw: Federated learning network object
        num_rounds: Number of federated learning rounds
        epochs_per_round: Number of local epochs per round
        experiment_name: Name for the experiment
    """
    await flnw.train(num_rounds=num_rounds, epochs_per_round=epochs_per_round, experiment_name=experiment_name)
    print(f"Experiment '{experiment_name}' completed.")

## Experiment Configuration

In [4]:
# Experiment parameters
alpha_values = [0.4, 0.8, 10]  # Dirichlet alpha values to test
network_types = ["centralised", "decentralised"]  # Network architectures
task_types = [
    {"name": "regression", "model": RegressionModel, "criterion": nn.MSELoss},
    {"name": "binary_classification", "model": BinaryClassifierModel, "criterion": nn.BCEWithLogitsLoss},
    {"name": "image_classification", "model": CNNModel, "criterion": nn.CrossEntropyLoss}
]

## Run Experiments
Choose a specific configuration to run by uncommenting and modifying the code below.

In [ ]:
# Example: Centralised Regression with Alpha = 0.01
# Change these variables to run different experiments

task_type = task_types[0]  # 0: regression, 1: binary_classification, 2: image_classification
network_type = network_types[0]  # 0: centralised, 1: decentralised
alpha = alpha_values[0]  # Choose from alpha_values list

# Create the configuration
config = {
    "network_name": f"{network_type}_{task_type['name']}_alpha_{alpha}",
    "network_type": network_type,
    "model_type": "parametric",
    "ml_framework": "pytorch",
    "comms": "async",
    "federated_rounds": 5,
    "network_size": 10,  # Fixed network size for Experiment 2
    "metrics": ["mean_perfomance", "convergence"],
    "logger": "mlflow",
    "node_base_config": {
        "model": None,
        "learning_rate": 1e-3,
        "optimizer": SGD,
        "train_data": None,
        "val_data": None,
        "criterion": task_type["criterion"],
        "random_sampling": False,
        "n_epochs": 5,
        "batch_size": 16
    },
    "experiment_config": {
        "task": task_type["name"],
        "partitioner": DirichletPartitioner,
        "alpha": alpha,  # Variable parameter
        "num_worker": 2 if task_type["name"] != "regression" else 1,
        "max_samples": 1000
    },
    "adjcency_matrix": None,
    "model": None
}

# Create the model
model = task_type["model"]()

# Create the experiment
print(f"Running experiment: {network_type} {task_type['name']} with alpha {alpha}")
flnw = create_experiment(config, model)

# Run the experiment
await execute_experiment(
    flnw, 
    num_rounds=5,  # Set to desired number (20 for full experiment)
    epochs_per_round=1, 
    experiment_name=f"exp2_{network_type}_{task_type['name']}_alpha_{alpha}"
)

In [ ]:
# Custom experiment runner - select specific combinations
# Uncomment and modify to run specific experiment combinations

async def run_selected_experiments(selected_task_types=None, selected_network_types=None, selected_alphas=None):
    """
    Run experiments with specific parameter combinations
    
    Args:
        selected_task_types: List of task indices to run (e.g., [0, 1] for regression and binary classification)
        selected_network_types: List of network type indices to run (e.g., [0] for centralised only)
        selected_alphas: List of alpha values to run (e.g., [0.01, 0.05])
    """
    if selected_task_types is None:
        selected_task_types = range(len(task_types))
    if selected_network_types is None:
        selected_network_types = range(len(network_types))
    if selected_alphas is None:
        selected_alphas = alpha_values
        
    experiment_count = 0
    total_experiments = len(selected_task_types) * len(selected_network_types) * len(selected_alphas)
    start_time = time.time()
    
    print(f"Starting selected experiments: {total_experiments} total experiments to run")
    
    # Loop through selected task types
    for task_idx in selected_task_types:
        task_type = task_types[task_idx]
        
        # Loop through selected network types
        for net_idx in selected_network_types:
            network_type = network_types[net_idx]
            
            print(f"\n{'='*80}")
            print(f"Running experiments for {network_type} {task_type['name']}")
            print(f"{'='*80}\n")
            
            # Loop through selected alpha values
            for alpha in selected_alphas:
                experiment_count += 1
                
                print(f"\nExperiment {experiment_count}/{total_experiments}")
                print(f"Task: {task_type['name']}, Network: {network_type}, Alpha: {alpha}")
                
                # Create configuration
                config = {
                    "network_name": f"{network_type}_{task_type['name']}_alpha_{alpha}",
                    "network_type": network_type,
                    "model_type": "parametric",
                    "ml_framework": "pytorch",
                    "comms": "async",
                    "federated_rounds": 5,
                    "network_size": 10,  # Fixed for Experiment 2
                    "metrics": ["mean_perfomance", "convergence"],
                    "logger": "mlflow",
                    "node_base_config": {
                        "model": None,
                        "learning_rate": 1e-3,
                        "optimizer": SGD,
                        "train_data": None,
                        "val_data": None,
                        "criterion": task_type["criterion"],
                        "random_sampling": False,
                        "n_epochs": 5,
                        "batch_size": 16
                    },
                    "experiment_config": {
                        "task": task_type["name"],
                        "partitioner": DirichletPartitioner,
                        "alpha": alpha,  # Variable parameter
                        "num_worker": 2 if task_type["name"] != "regression" else 1,
                        "max_samples": 1000
                    },
                    "adjcency_matrix": None,
                    "model": None
                }
                
                try:
                    # Create model
                    model = task_type["model"]()
                    
                    # Create experiment
                    print(f"Creating experiment: {network_type} {task_type['name']} with alpha {alpha}")
                    flnw = create_experiment(config, model)
                    
                    # Execute experiment
                    print(f"Running experiment...")
                    await execute_experiment(
                        flnw, 
                        num_rounds=5,  # Set to 20 for full experiment, 5 for testing
                        epochs_per_round=1,
                        experiment_name=f"exp2_{network_type}_{task_type['name']}_alpha_{alpha}"
                    )
                    
                except Exception as e:
                    print(f"Error in experiment {experiment_count}: {e}")
                
                # Wait briefly between experiments
                await asyncio.sleep(2)
    
    total_time = time.time() - start_time
    print(f"\n{'='*80}")
    print(f"Selected experiments completed!")
    print(f"Total time: {total_time:.1f} seconds")
    print(f"{'='*80}\n")

# Example usage:
# Run only regression task with both network types but only alpha values 0.01 and 0.05
# await run_selected_experiments(
#     selected_task_types=[0],  # 0: regression
#     selected_network_types=[0, 1],  # Both centralised and decentralised
#     selected_alphas=[0.01, 0.05]  # Only smaller alpha values
# )

# Run all tasks but only for centralised networks with alpha 0.01
# await run_selected_experiments(
#     selected_network_types=[0],  # 0: centralised
#     selected_alphas=[0.01]  # Only alpha 0.01
# )

## Automated Experiment Loop
This cell runs all experiments for a specific network type and task type.
Uncomment and modify to run a batch of experiments.

In [ ]:
# Run all experiments for all task types and network types
import asyncio
import time
import datetime

async def run_all_experiments():
    """Run all alpha parameter experiments for all task types and network architectures"""
    
    experiment_count = 0
    total_experiments = len(task_types) * len(network_types) * len(alpha_values)
    start_time = time.time()
    
    print(f"Starting all experiments: {total_experiments} total experiments to run")
    print(f"Start time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    
    # Loop through all task types
    for task_idx, task_type in enumerate(task_types):
        # Loop through all network types
        for net_idx, network_type in enumerate(network_types):
            print(f"\n{'='*80}")
            print(f"Running experiments for {network_type} {task_type['name']}")
            print(f"{'='*80}\n")
            
            # Loop through all alpha values
            for alpha_idx, alpha in enumerate(alpha_values):
                experiment_count += 1
                
                print(f"\nExperiment {experiment_count}/{total_experiments}")
                print(f"Task: {task_type['name']}, Network: {network_type}, Alpha: {alpha}")
                
                # Create configuration
                config = {
                    "network_name": f"{network_type}_{task_type['name']}_alpha_{alpha}",
                    "network_type": network_type,
                    "model_type": "parametric",
                    "ml_framework": "pytorch",
                    "comms": "async",
                    "federated_rounds": 5,
                    "network_size": 10,  # Fixed for Experiment 2
                    "metrics": ["mean_perfomance", "convergence"],
                    "logger": "mlflow",
                    "node_base_config": {
                        "model": None,
                        "learning_rate": 1e-3,
                        "optimizer": SGD,
                        "train_data": None,
                        "val_data": None,
                        "criterion": task_type["criterion"],
                        "random_sampling": False,
                        "n_epochs": 5,
                        "batch_size": 16
                    },
                    "experiment_config": {
                        "task": task_type["name"],
                        "partitioner": DirichletPartitioner,
                        "alpha": alpha,  # Variable parameter
                        "num_worker": 2 if task_type["name"] != "regression" else 1,
                        "max_samples": 1000
                    },
                    "adjcency_matrix": None,
                    "model": None
                }
                
                try:
                    # Create model
                    model = task_type["model"]()
                    
                    # Create experiment
                    print(f"Creating experiment: {network_type} {task_type['name']} with alpha {alpha}")
                    flnw = create_experiment(config, model)
                    
                    # Execute experiment
                    print(f"Running experiment...")
                    await execute_experiment(
                        flnw, 
                        num_rounds=5,  # Set to 20 for full experiment, 5 for testing
                        epochs_per_round=1,
                        experiment_name=f"exp2_{network_type}_{task_type['name']}_alpha_{alpha}"
                    )
                    
                    # Calculate elapsed time
                    elapsed = time.time() - start_time
                    avg_time_per_exp = elapsed / experiment_count
                    remaining = avg_time_per_exp * (total_experiments - experiment_count)
                    
                    # Print progress
                    print(f"\nCompleted experiment {experiment_count}/{total_experiments}")
                    print(f"Elapsed time: {elapsed:.1f} seconds")
                    print(f"Estimated time remaining: {remaining:.1f} seconds")
                    print(f"Estimated completion time: {datetime.datetime.now() + datetime.timedelta(seconds=remaining)}")
                    
                except Exception as e:
                    print(f"Error in experiment {experiment_count}: {e}")
                
                # Wait briefly between experiments
                await asyncio.sleep(2)
    
    # Print completion summary
    total_time = time.time() - start_time
    print(f"\n{'='*80}")
    print(f"All experiments completed!")
    print(f"Total time: {total_time:.1f} seconds")
    print(f"End time: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print(f"{'='*80}\n")


In [6]:
await run_all_experiments()

Starting all experiments: 18 total experiments to run
Start time: 2025-09-09 08:04:46

Running experiments for centralised regression


Experiment 1/18
Task: regression, Network: centralised, Alpha: 0.4
Creating experiment: centralised regression with alpha 0.4


/home/sgtec/miniconda3/envs/moxifl/lib/python3.11/site-packages/flwr_datasets/utils.py:109: UserWarning: The currently tested dataset are ['mnist', 'ylecun/mnist', 'cifar10', 'uoft-cs/cifar10', 'fashion_mnist', 'zalando-datasets/fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet', 'scikit-learn/adult-census-income', 'cifar100', 'uoft-cs/cifar100', 'svhn', 'ufldl-stanford/svhn', 'sentiment140', 'stanfordnlp/sentiment140', 'speech_commands', 'LIUM/tedlium', 'flwrlabs/femnist', 'flwrlabs/ucf101', 'flwrlabs/ambient-acoustic-context', 'jlh/uci-mushrooms', 'Mike0307/MNIST-M', 'flwrlabs/usps', 'scikit-learn/iris', 'flwrlabs/pacs', 'flwrlabs/cinic10', 'flwrlabs/caltech101', 'flwrlabs/office-home', 'flwrlabs/fed-isic2019']. Given: SIH/palmer-penguins.
  warnings.warn(
/home/sgtec/miniconda3/envs/moxifl/lib/python3.11/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (10) was not satisfied for alpha ([np.float64(0.4)]) after

Running experiment...
Training Complete!
Experiment 'exp2_centralised_regression_alpha_0.4' completed.

Completed experiment 1/18
Elapsed time: 19.7 seconds
Estimated time remaining: 335.1 seconds
Estimated completion time: 2025-09-09 08:10:41.676126
Training Complete!
Experiment 'exp2_centralised_regression_alpha_0.4' completed.

Completed experiment 1/18
Elapsed time: 19.7 seconds
Estimated time remaining: 335.1 seconds
Estimated completion time: 2025-09-09 08:10:41.676126

Experiment 2/18
Task: regression, Network: centralised, Alpha: 0.8
Creating experiment: centralised regression with alpha 0.8

Experiment 2/18
Task: regression, Network: centralised, Alpha: 0.8
Creating experiment: centralised regression with alpha 0.8


/home/sgtec/miniconda3/envs/moxifl/lib/python3.11/site-packages/flwr_datasets/utils.py:109: UserWarning: The currently tested dataset are ['mnist', 'ylecun/mnist', 'cifar10', 'uoft-cs/cifar10', 'fashion_mnist', 'zalando-datasets/fashion_mnist', 'sasha/dog-food', 'zh-plus/tiny-imagenet', 'scikit-learn/adult-census-income', 'cifar100', 'uoft-cs/cifar100', 'svhn', 'ufldl-stanford/svhn', 'sentiment140', 'stanfordnlp/sentiment140', 'speech_commands', 'LIUM/tedlium', 'flwrlabs/femnist', 'flwrlabs/ucf101', 'flwrlabs/ambient-acoustic-context', 'jlh/uci-mushrooms', 'Mike0307/MNIST-M', 'flwrlabs/usps', 'scikit-learn/iris', 'flwrlabs/pacs', 'flwrlabs/cinic10', 'flwrlabs/caltech101', 'flwrlabs/office-home', 'flwrlabs/fed-isic2019']. Given: SIH/palmer-penguins.
  warnings.warn(
/home/sgtec/miniconda3/envs/moxifl/lib/python3.11/site-packages/flwr_datasets/partitioner/dirichlet_partitioner.py:300: UserWarning: The specified min_partition_size (10) was not satisfied for alpha ([np.float64(0.8)]) after

Running experiment...
Training Complete!
Experiment 'exp2_centralised_regression_alpha_0.8' completed.

Completed experiment 2/18
Elapsed time: 35.2 seconds
Estimated time remaining: 281.8 seconds
Estimated completion time: 2025-09-09 08:10:03.949303
Training Complete!
Experiment 'exp2_centralised_regression_alpha_0.8' completed.

Completed experiment 2/18
Elapsed time: 35.2 seconds
Estimated time remaining: 281.8 seconds
Estimated completion time: 2025-09-09 08:10:03.949303

Experiment 3/18
Task: regression, Network: centralised, Alpha: 10
Creating experiment: centralised regression with alpha 10

Experiment 3/18
Task: regression, Network: centralised, Alpha: 10
Creating experiment: centralised regression with alpha 10


...

Running experiment...
Training Complete!
Experiment 'exp2_decentralised_image_classification_alpha_0.4' completed.

Completed experiment 16/18
Elapsed time: 671.3 seconds
Estimated time remaining: 83.9 seconds
Estimated completion time: 2025-09-09 08:17:22.038451
Training Complete!
Experiment 'exp2_decentralised_image_classification_alpha_0.4' completed.

Completed experiment 16/18
Elapsed time: 671.3 seconds
Estimated time remaining: 83.9 seconds
Estimated completion time: 2025-09-09 08:17:22.038451

Experiment 17/18
Task: image_classification, Network: decentralised, Alpha: 0.8
Creating experiment: decentralised image_classification with alpha 0.8
Loading federated CIFAR-10 dataset: uoft-cs/cifar10

Experiment 17/18
Task: image_classification, Network: decentralised, Alpha: 0.8
Creating experiment: decentralised image_classification with alpha 0.8
Loading federated CIFAR-10 dataset: uoft-cs/cifar10


2025/09/09 08:16:02 INFO mlflow.tracking.fluent: Experiment with name 'exp2_decentralised_image_classification_alpha_0.8' does not exist. Creating a new experiment.


Running experiment...
Training Complete!
Experiment 'exp2_decentralised_image_classification_alpha_0.8' completed.

Completed experiment 17/18
Elapsed time: 779.7 seconds
Estimated time remaining: 45.9 seconds
Estimated completion time: 2025-09-09 08:18:32.463181
Training Complete!
Experiment 'exp2_decentralised_image_classification_alpha_0.8' completed.

Completed experiment 17/18
Elapsed time: 779.7 seconds
Estimated time remaining: 45.9 seconds
Estimated completion time: 2025-09-09 08:18:32.463181

Experiment 18/18
Task: image_classification, Network: decentralised, Alpha: 10
Creating experiment: decentralised image_classification with alpha 10
Loading federated CIFAR-10 dataset: uoft-cs/cifar10

Experiment 18/18
Task: image_classification, Network: decentralised, Alpha: 10
Creating experiment: decentralised image_classification with alpha 10
Loading federated CIFAR-10 dataset: uoft-cs/cifar10


2025/09/09 08:17:50 INFO mlflow.tracking.fluent: Experiment with name 'exp2_decentralised_image_classification_alpha_10' does not exist. Creating a new experiment.


Running experiment...
Training Complete!
Experiment 'exp2_decentralised_image_classification_alpha_10' completed.

Completed experiment 18/18
Elapsed time: 887.7 seconds
Estimated time remaining: 0.0 seconds
Estimated completion time: 2025-09-09 08:19:34.599294
Training Complete!
Experiment 'exp2_decentralised_image_classification_alpha_10' completed.

Completed experiment 18/18
Elapsed time: 887.7 seconds
Estimated time remaining: 0.0 seconds
Estimated completion time: 2025-09-09 08:19:34.599294

All experiments completed!
Total time: 889.7 seconds
End time: 2025-09-09 08:19:36


All experiments completed!
Total time: 889.7 seconds
End time: 2025-09-09 08:19:36

